## TLDR
Python's `random` library is insecure, and in certain cases, by knowing few of the pseudorandom numbers generated by it, you can guess any other number generated by it. The PoC below shows how to solve for random.randrange($2^{37}$).

Instead of needing all 624 Mersenne Twister states, we get all the even states, 3 times in a row. This gives enough information to fully calculate the missing states.
When obtaining new MT states from the old states, the twisting function is applied on the old states and it looks similar to this:

`state[new][i]=function(1st bit of state[old][i], 31 bits of state[old][(i+1) % 624], 32 bits of state[old][(i+397) % 624])`

## Bit longer TLDR
### Note: everytime I write random I mean pseudorandom, cuz nothing is truly random

* A warning is shown when visiting Python's [`random` library documentation](https://docs.python.org/3/library/random.html):
> **Warning:** The pseudo-random generators of this module should not be used for security purposes. For security or cryptographic uses, see the `secrets` module.

* Its PRNG uses  Mersenne Twister 19937, which is based on prime $2^{19937} - 1$.

* Its periodicity is $2^{19937}$, so you just need generate $2^{19937}$ values and then it will start repeating the generated numbers (gl with that).

* More critically, based on its seed, it uses 624 states to generate the random numbers.

* To simplify, when you ask for the first random number, it takes the first state `state[0]`, applies a tampering function on it and returns ```tamper(state[0])```. This tampering function is fully reversible. When you ask for the next number, it does the same with `state[1]` etc.

* Once you've asked for 624 random numbers, it applies a different reversible function on its 624 states, to obtain 624 new states.

Few resources to learn more about this:
- A short video from SANS explaining Mersenne Twisters and their vulnerabilities: https://www.youtube.com/watch?v=Jo5Nlbqd-Vg
- Some blog posts explaining the Mersenne Twister used by Python's random module: https://jazzy.id.au/2010/09/22/cracking_random_number_generators_part_3.html and https://jazzy.id.au/2010/09/25/cracking_random_number_generators_part_4.html


Almost all of the examples that I've found online that can be used to predict numbers generated by Python's `random` module used `random.randrange(0xFFFFFFFF)` as the function used to generate random numbers. `0xFFFFFFFF` is just $2^{32}$-1, or around 4 billion. 

Initially my CTF challenge used it too, but then I realized one unintended solution would be to keep guessing the same number all the time, and there's a 1 in $2^{32}$ chances you will guess it. On average, you would guess it in around 2 billion tries.

So I changed the challenge to use a slightly larger number i.e. `random.randrange(10E9,10E10)`, thinking it would be as easy to solve without the unintended bruteforcing solution:D (spoiler alert: now the challenge is either unsolvable, or a lot more complicated to solve, and I haven't found a way yet, sorrrrrrrry :DDDD)

### Let's look at how the `randrange()` function works.
By looking at the Python source code of the `random` module at e.g. `/usr/lib/python3.7/random.py` we can see
`random.randrange(10E9,10E10)` will call `random._randbelow(10E10 - 10E9)`, or `random._randbelow(9E10)`. A simplified version of the function can be seen below:

```
_randbelow(n):
    k = n.bit_length()  
    r = getrandbits(k)        
    while r >= n:
        r = getrandbits(k)
    return r
```

This calls the built-in (aka C Python) method  `_random.Random().getrandbits(int(9E10).bit_length())`, or `getrandbits(37)`, which calls the CPython function `genrand_int32()` two times. See e.g. https://github.com/python/cpython/blob/3.7/Modules/_randommodule.c
  
 This is basically equivalent to calling `getrandbits(32)` twice, and keeping the entire 32 bits of the first number and the 5 Most Significant Bits (MSB) of the second number.
 
  The 2 numbers are combined in a 37 bit number that looks like: `5MSBOfSecondRandomNumber` concatenated to `32bitsOfFirstRandomNumber`.
  
  If this 37 bit number is less than 9E10, what is returned is is 10E9 + (`5MSBOfSecondRandomNumber` concatenated to `32bitsOfFirstRandomNumber`)
  
  The problem with solving the challenge is the code below. If this 37 bit number is larger than or equal to 9E10, the 37 bit number is discarded and the procedure described above is repeated (i.e. `genrand_int32()` called again two times etc.
```
while r >= n:
    r = getrandbits(k)
```
  This makes building a sequential series of Mersenne Twister states very difficult when using `random.randrange(n)`, if `n` is  much smaller than 2<sup>n.bit_length()</sup> - 1. That's the reason most online crackers use `randrange(0xFFFFFFFF)`. You could of course be lucky and get a large series of numbers that are all less than `n`, but you will need to run `random.randrange(n)` many times, and I haven't looked at the statistics of that.

  For this reason, in the PoC below I changed the `random.randrange(10E9,10E10)` to `random.randrange(10E9,10E9 + ` $2^{37} - 1$).
  This guarantees `random._randbelow(`$2^{37}-1$) is called instead, where $2^{37}-1$ is the largest 37 bit number.
  
## The PoC below shows how to solve for `randrange(`$2^{37}$).
Further additions could be:
* Generalize it so it can be used to solve for any `randrange`($2^x$), for `x>=32`.
* Figure out the limits of `n` for solving `randrange(n)`, for `n < `$2^x$.
* Figure out the limits of `x` for solving `randrange(`$2^x$), for `x<32`.


In [1]:
#!/usr/bin/env python3

# fc: Most of the functions here are taken from https://github.com/tliston/mt19937/blob/main/mt19937.py

import random

# this is simply a python implementation of a standard Mersenne Twister PRNG.
# the parameters used, implement the MT19937 variant of the PRNG, based on the
# Mersenne prime 2^19937−1
# see https://en.wikipedia.org/wiki/Mersenne_Twister for a very good explanation
# of the math behind this...

class mt19937():
    #fc: some constants used for python's Mersenne Twister
    u, d = 11, 0xFFFFFFFF
    s, b = 7, 0x9D2C5680
    t, c = 15, 0xEFC60000
    l = 18
    n = 624 # number of states
    m = 397 # one of the state indexes used when twisting states
    a = 0x9908B0DF # magic number used when twisting states
    w = 32
    r = 31
    f = 1812433253
    lower_mask = (1 << r) - 1
    upper_mask = (~lower_mask) & 0xFFFFFFFF

    
    # fc: this function returns the last 32 bits of a supplied integer
    def my_int32(self, x):
        return(x & 0xFFFFFFFF)

    def __init__(self, seed):

        self.MT = [0] * self.n
        self.index = self.n + 1
        self.MT[0] = self.my_int32(seed)
        for i in range(1, self.n):
            self.MT[i] = self.my_int32((self.f * (self.MT[i - 1] ^ (self.MT[i - 1] >> (self.w - 2))) + i))

    # fc: basically equivalent to getrandbits(32)
    def extract_number(self):
        if self.index >= self.n:
            self.twistStates()
            self.index = 0
        y = self.MT[self.index]
        y = self.temper(y)
        self.index += 1
        return self.my_int32(y)
    
    # fc: this just modifies one of the 624 states in order to make it more statistically pretty
    # this function is reversible and is implemented in untemper()
    def temper(self, y):
        # this implements the so-called "tempering matrix"
        # this, functionally, should alter the output to
        # provide a better, higher-dimensional distribution
        # of the most significant bits in the numbers extracted
        y = y ^ ((y >> self.u) & self.d)
        y = y ^ ((y << self.s) & self.b)
        y = y ^ ((y << self.t) & self.c)
        y = y ^ (y >> self.l)
        return y

    # fc: This changes the current 624 states to their next values
    # this function is also reversible, but haven't implemented its reverse
    def twistStates(self): 
        for i in range(0, self.n):
            x = (self.MT[i] & self.upper_mask) + (self.MT[(i + 1) % self.n] & self.lower_mask)
            xA = x >> 1
            if(x % 2) != 0:
                xA = xA ^ self.a
            self.MT[i] = self.MT[(i + self.m) % self.n] ^ xA
    

# fc: the reverse of temper()
# basically untemper(temper(x)) = x
def untemper(y):
    y ^= y >> mt19937.l
    y ^= y << mt19937.t & mt19937.c
    for i in range(7):
        y ^= y << mt19937.s & mt19937.b
    for i in range(3):
        y ^= y >> mt19937.u & mt19937.d
    return y

##### fc: This function calculates the next state based on the previous states and the twisting function
##### Basically state[1][i] = function (state[0][i],state[0][(i+1) %624],state[0][(i+397) %624])
def getNextState_static(state_i,state_i_plus_1,state_i_plus_397):
    # get the first bit of the current number,  and the last 31 bits of the next number
    x = (state_i & mt19937.upper_mask) + (state_i_plus_1 & mt19937.lower_mask)
    # bitshift it by 1 to the right
    xA = x >> 1
    # if x is odd, xor with magic number
    if(x % 2) != 0:
        xA = xA ^ mt19937.a
    # xor it with the 397th next number
    return state_i_plus_397 ^ xA

##### fc: This function calculates the next state based on the previous states and the twisting function
##### Basically state[1][i] = function (state[0][i],state[0][(i+1) %624],state[0][(i+397) %624])
##### Only the 1st bit of state[0][i] is used, so this function is used to bruteforce that bit
def getNextState_unknownFirstBit(states, j, i, firstbit):
    # if i+1>623, we've reached the end of the states, make sure we use the next states
    if (i + 1) > 623:
        # we don't know the first bit of the current number (so we bruteforce it), but we know the last 31 bits of the next number
        x = (firstbit * mt19937.upper_mask) + (states[j+1][(i + 1) % mt19937.n] & mt19937.lower_mask)
    else:
        x = (firstbit * mt19937.upper_mask) + (states[j][(i + 1) % mt19937.n] & mt19937.lower_mask)

    # bitshift it by 1 to the right
    xA = x >> 1
    # if x is odd, XOR with magic number
    if(x % 2) != 0:
        xA = xA ^ mt19937.a
    # XOR it with the 397th next number
    # if i + 397 > 623, we've reached the end of the states, make sure we use the next states (which at this point should already be calculated)
    if (i + mt19937.m) > 623:
        return states[j+1][(i + mt19937.m) % mt19937.n] ^ xA
    else:
        return states[j][(i + mt19937.m) % mt19937.n] ^ xA

In [2]:
random.seed(4710924109828301) # Any seed should work, this was the one used in the challenge

# Now we would interact with the server to extract enough random numbers to partially build 3x 624 states
# Since one random.randrange(10E9,10E9 + 2**37 - 1) basically calls random.getrandbits(32) twice, we need to call it 312 times for it to go through all 624 states

# First we initialize 3 "empty" arrays. partialStates[0] will be an array of the first 624 MT states, partialStates[1] will be an array of the next 624 states etc.
partialStates = [[-1] * mt19937.n, [-1] * mt19937.n, [-1] * mt19937.n]  # we initialize the states with invalid value -1

for j in range(3):
    for i in range(0,mt19937.n,2):
        nextRandomReceivedFromServer=random.randrange(10E9,10E9 + 2**37 - 1) # These would be the random numbers received from the server
        tempered = int(nextRandomReceivedFromServer - 10E9) & 0xFFFFFFFF # we want only the last 32 bits of the first random number, which is equivalent to running random.getrandbits(32) once. We basically discard the MSB 5 bits of the second call to random.getrandbits(32) in order to make this approach more general 
        partialStates[j][i] = untemper(tempered) # we will thus obtain all the even states e.g. state[0],state[2],state[4],...,state[622]

        
############################################
# Let's finish filling the missing uneven states
############################################

# partialStates[1][0] is a function of 1st bit of partialStates[0][0], last 31 bits of partialStates[0][1] and all of partialStates[0][0+397]
# since we have all even states, we have partialStates[1][0], partialStates[0][0], but miss partialStates[0][1] and partialStates[0][397]. we are missing 31 + 32 = 63 bits
# we're missing 63 bits, so let's skip this one for the moment

# partialStates[1][1] is a function of 1st bit of partialStates[0][1], last 31 bits of partialStates[0][2] and all of partialStates[0][1+397]
# since we have all even states, we have partialStates[0][2] and partialStates[0][398] but miss partialStates[0][1]. we are missing 1 bit
# as we're only missing 1 bit, we can "bruteforce" it 

partialStates[1][1]=(getNextState_unknownFirstBit(states=partialStates,j=0,i=1,firstbit=0), getNextState_unknownFirstBit(states=partialStates,j=0,i=1,firstbit=1))
print(partialStates[1][1]) # so it's either of these 2 values

# how can we identify which of the 2 is correct? we'll use partialStates[2][0], which we know
# partialStates[2][0] is a function of 1st bit of partialStates[1][0], last 31 bits of partialStates[1][1] and all of partialStates[1][0+397]
# since we have all even states, we have partialStates[2][0], partialStates[1][0]. We also know partialStates[1][1] is one of 2 values (calculated above). Let's calculate partialStates[1][397]

# partialStates[1][397] is a function of 1st bit of partialStates[0][397], last 31 bits of partialStates[0][398] and all of partialStates[1][(397+397) % 624]. 
# we have partialStates[0][398] and we have partialStates[1][170], we can "bruteforce" the missing bit of partialStates[0][397]
partialStates[1][397]=(getNextState_unknownFirstBit(states=partialStates,j=0,i=397,firstbit=0), getNextState_unknownFirstBit(states=partialStates,j=0,i=397,firstbit=1))
print(partialStates[1][397]) # so it's either of these 2 values

# now we have 2 possible values for partialStates[1][1] and 2 possible values for partialStates[1][397]. let's find which are the correct ones
# there is a 1 in 2**62 chance this is incorrect, but we'll take our chances :D
for state_i_plus_1 in partialStates[1][1]:
    for state_i_plus_397 in partialStates[1][397]:
        if partialStates[2][0]==getNextState_static(state_i=partialStates[1][0],state_i_plus_1=state_i_plus_1,state_i_plus_397=state_i_plus_397):
            print("partialStates[1][1] = {}, partialStates[1][397] = {}".format(state_i_plus_1,state_i_plus_397))
            partialStates[1][1]=state_i_plus_1
            partialStates[1][397]=state_i_plus_397
            found=True
    if found:
        break
        
############################################        
# Now that we know how to calculate the uneven states, let's calculate all of them
############################################

j=1
for i in range(0,mt19937.n,2):
    if partialStates[1][i+1]!= -1: # if we've already calculated it e.g. reached partialStates[1][397]
        continue
        
    partialStates[1][i+1]=(getNextState_unknownFirstBit(states=partialStates,j=0,i=i+1,firstbit=0), getNextState_unknownFirstBit(states=partialStates,j=0,i=i+1,firstbit=1))
#     print(partialStates[1][i]) # debugging
    # if i+397>623, we don't need to calculate it as we know it already, remove it from for loop too
    if (i + 397 > 623 and j==1): 
        j+=1
    partialStates[j][(i+397) % 624]=(getNextState_unknownFirstBit(states=partialStates,j=j-1,i=(i+397) % 624,firstbit=0), getNextState_unknownFirstBit(states=partialStates,j=j-1,i=(i+397) % 624,firstbit=1))
#     print(partialStates[1][i+397]) # debugging

    found = False
    for state_i_plus_1 in partialStates[1][i+1]:
        for state_i_plus_397 in partialStates[j][(i+397) % 624]:
            if partialStates[2][i]==getNextState_static(state_i=partialStates[1][i],state_i_plus_1=state_i_plus_1,state_i_plus_397=state_i_plus_397):
                print("partialStates[1][{}] = {}, partialStates[{}][{}] = {}".format(i+1,state_i_plus_1,j,(i+397) % 624,state_i_plus_397)) # debugging
                partialStates[1][i+1]=state_i_plus_1
                partialStates[j][(i+397) % 624]=state_i_plus_397
                found=True
        if found:
            break
    if not found: # we shouldn't reach this
        print("Error in the algorithm (or, less likely, the 1 in 2**62 chance was hit)")
        
        
        


(2653792599, 3727534423)
(3134592619, 4208334443)
partialStates[1][1] = 2653792599, partialStates[1][397] = 3134592619
partialStates[1][3] = 1112079699, partialStates[1][399] = 1562263300
partialStates[1][5] = 2619809924, partialStates[1][401] = 832692296
partialStates[1][7] = 176311172, partialStates[1][403] = 1814465511
partialStates[1][9] = 1487211524, partialStates[1][405] = 2786181027
partialStates[1][11] = 589134402, partialStates[1][407] = 4089596404
partialStates[1][13] = 2391278992, partialStates[1][409] = 1810260917
partialStates[1][15] = 4170586443, partialStates[1][411] = 291706931
partialStates[1][17] = 3868091044, partialStates[1][413] = 1345442081
partialStates[1][19] = 3923000326, partialStates[1][415] = 1351725747
partialStates[1][21] = 1583070130, partialStates[1][417] = 3178317475
partialStates[1][23] = 3990028939, partialStates[1][419] = 1917738496
partialStates[1][25] = 2485773068, partialStates[1][421] = 3031335943
partialStates[1][27] = 1814071842, partialStates[

In [3]:
############################################
# Now we have the entire 624 states in partialStates[1]
# We can initialize our own version of an MT19937 PRNG.
############################################
solvedPRNG = mt19937(0)

# Create our MT19937 PRNG clone
print("Initialize our MT19937 PRNG clone based on the partialStates[1] we got from the random server and further calculated by us")
for i in range(mt19937.n):
    solvedPRNG.MT[i] = partialStates[1][i]
# By this point, the random server application is already using states[3] so we need to move our states there as well
print("Twisting states")
solvedPRNG.twistStates()
solvedPRNG.twistStates()
solvedPRNG.index=0

# check to make sure our cloning worked...
count=1000 # let's guess the next 1000 random numbers
print("Now, we'll test the clone...")
print("\nPython       Our clone")
for i in range(count):
    nextRandomFromServer=random.randrange(10E9,10E9 + 2**37 - 1) # These would be the random numbers generated by the server
    ourRandomPart1 = solvedPRNG.extract_number() # we need all 32 bits of this one
    ourRandomPart2 = solvedPRNG.extract_number() # we only need the MSB 5 bits of this one
    ourRandomNumber = (ourRandomPart2 >> 32-5 << 32) + ourRandomPart1
    ourRandomGuess = int(ourRandomNumber + 10E9)
    print("%12.12i - %12.12i (%r)" % (nextRandomFromServer, ourRandomGuess, (nextRandomFromServer == ourRandomGuess)))

Initialize our MT19937 PRNG clone based on the partialStates[1] we got from the random server and further calculated by us
Twisting states
Now, we'll test the clone...

Python       Our clone
080196979509 - 080196979509 (True)
084207899254 - 084207899254 (True)
044890871295 - 044890871295 (True)
107801271675 - 107801271675 (True)
056281174346 - 056281174346 (True)
031982490588 - 031982490588 (True)
089827608393 - 089827608393 (True)
094954699751 - 094954699751 (True)
105271519335 - 105271519335 (True)
041205341666 - 041205341666 (True)
091897446733 - 091897446733 (True)
065562688034 - 065562688034 (True)
123476368165 - 123476368165 (True)
030121874257 - 030121874257 (True)
061034973263 - 061034973263 (True)
097889051196 - 097889051196 (True)
128999836801 - 128999836801 (True)
098962786699 - 098962786699 (True)
014475441346 - 014475441346 (True)
059286031759 - 059286031759 (True)
071984144432 - 071984144432 (True)
070711578387 - 070711578387 (True)
046917384364 - 046917384364 (True)
115